# Simple Facial Detection Algorithm in OpenCV
A computer program that decides whether an image is a positive image (face image)
or negative image (non-face image) is called a **classifier**.
A classifier is trained on hundreds of thousands of face and non-face images to
learn how to classify a new image correctly.
OpenCV provides us with two pre-trained and ready to be used for face detection classifiers
(good description of these classifiers and how they compare can be found
[here](https://www.superdatascience.com/opencv-face-detection/)):

1. [Haar Classifier](https://docs.opencv.org/3.3.0/d7/d8b/tutorial_py_face_detection.html)
2. Local Binary Patterns (LBP) Classifier

Both of these classifiers process images in grayscale,
basically because we don't need color information to decide if a picture has a face or not.
As these are pre-trained in OpenCV, their learned knowledge files
also come bundled with OpenCV in XML files stored in `..opencv-3.3.0/data/haarcascades/` and
`..opencv-3.3.0/data/lbpcascades/` folders.

* https://www.superdatascience.com/opencv-face-detection/
* Rodrigo Agundez - Building a live face recognition system in the blink of a very slow eye - https://www.youtube.com/watch?v=MDaZtJPv3Ik
* https://github.com/rragundez/PyData

We use `v2.CascadeClassifier.detectMultiScale()` to find faces or eyes,
and it is defined like this:

`cv2.CascadeClassifier.detectMultiScale(image[, scaleFactor[, minNeighbors[, flags[, minSize[, maxSize]]]]])`

Where the parameters are:

* `image`: Matrix of the [type CV_8U][01] containing an image where objects are detected.
* `scaleFactor`: Parameter specifying how much the image size is reduced at each image scale. This scale factor is used to create [scale pyramid][02]. Suppose, the scale factor is 1.03, it means we're using a small step for resizing, i.e. reduce size by 3%, we increase the chance of a matching size with the model for detection is found, while it's expensive.
* `minNeighbors`: Parameter specifying how many neighbors each candidate rectangle should have to retain it. This parameter will affect the quality of the detected faces: higher value results in less detections but with higher quality.
* `flags`: Parameter with the same meaning for an old cascade as in the function cvHaarDetectObjects. It is not used for a new cascade.
* `minSize`: Minimum possible object size. Objects smaller than that are ignored.
* `maxSize`: Maximum possible object size. Objects larger than that are ignored.

[01]:http://dovgalecs.com/blog/opencv-matrix-types/
[02]:https://docs.opencv.org/3.3.0/d4/d1f/tutorial_pyramids.html

In [1]:
import cv2
import sys
import logging as log
import datetime as dt
from time import sleep

#setting up the haar cascade classifiers from the opencv installation
facePath = "/home/jeff/src/opencv-3.3.0/data/haarcascades/haarcascade_frontalface_default.xml"
eyePath = "/home/jeff/src/opencv-3.3.0/data/haarcascades/haarcascade_eye.xml"
faceCascade = cv2.CascadeClassifier(facePath)
eyeCascade = cv2.CascadeClassifier(eyePath)

log.basicConfig(filename='/home/jeff/tmp/webcam.log', level=log.INFO)

video_capture = cv2.VideoCapture(0)
anterior = 0

In [2]:
while True:
    if not video_capture.isOpened():
        print('Unable to load camera.')
        sleep(5)
        pass

    # Capture frame-by-frame
    ret, frame = video_capture.read()

    # convert frame to gray scale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # find the faces
    faces = faceCascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    if anterior != len(faces):
        anterior = len(faces)
        log.info("faces: "+str(len(faces))+" at "+str(dt.datetime.now()))

    #for each face, find the eyes, and draw a box around the eyes
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        eyes = eyeCascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (225, 0, 0), 1)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    #stop reading frames if you press 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    # Display the resulting frame
    cv2.imshow('Video', frame)

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()

error: /home/jeff/src/opencv-3.4.0/modules/objdetect/src/cascadedetect.cpp:1698: error: (-215) !empty() in function detectMultiScale


# Sources
* [Face Detection in Python Using a Webcam](https://realpython.com/blog/python/face-detection-in-python-using-a-webcam/)
* [FACE DETECTION USING OPENCV AND PYTHON: A BEGINNER’S GUIDE](https://www.superdatascience.com/opencv-face-detection/)
* [Face Recognition With Python, in Under 25 Lines of Code](https://realpython.com/blog/python/face-recognition-with-python/)
* [FACE RECOGNITION USING OPENCV AND PYTHON: A BEGINNER’S GUIDE](https://www.superdatascience.com/opencv-face-recognition/)
* [Face detection with OpenCV and deep learning](https://www.pyimagesearch.com/2018/02/26/face-detection-with-opencv-and-deep-learning/)

* [A Practical Guide To Using Face Technology (Part I)](https://medium.com/iotforall/a-practical-guide-to-using-face-technology-part-i-7fe3fdb1bc4f)
* [Part II ???]()
* [Raspberry Pi: Facial landmarks + drowsiness detection with OpenCV and dlib](https://www.pyimagesearch.com/2017/10/23/raspberry-pi-facial-landmarks-drowsiness-detection-with-opencv-and-dlib/)
* [Drowsiness detection with OpenCV](https://www.pyimagesearch.com/2017/05/08/drowsiness-detection-opencv/)
* [Remote video streaming with face detection](https://medium.com/@QuantuMobile/remote-video-streaming-with-face-detection-d52ce2d71419)
* [Real-time face detection using OpenCV, Node.js, and WebSockets](http://drejkim.com/blog/2014/12/02/real-time-face-detection-using-opencv-nodejs-and-websockets/)
* [RasPi + OpenCV = Face Tracking](http://www.instructables.com/id/RasPi-OpenCV-Face-Tracking/)
* [Face Recognition on the Command-Line](https://github.com/ageitgey/face_recognition?utm_source=DigitalOcean_Newsletter)
* [Raspberry Pi Facial Recognition](https://www.hackster.io/gr1m/raspberry-pi-facial-recognition-16e34e?utm_source=hackster&utm_medium=email&utm_campaign=new_projects)
* [Smile Detection with Raspberry Pi Using OpenCV and Python](https://github.com/DeligenceTechnologies/Smile-detection-with-raspberry-pi-using-opencv-and-python)